In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 120)

LISTINGS = "../data/processed/listings_clean.csv"
CALENDAR = "../data/raw/calendar.csv.gz"

listings = pd.read_csv(LISTINGS)
listings.head(2), listings.shape


In [ ]:
cal = pd.read_csv(CALENDAR, low_memory=False, parse_dates=["date"])
cal.head(2), cal.shape, cal.columns.tolist()


In [ ]:
cal["date"] = pd.to_datetime(cal["date"], errors="coerce")
cal["available"] = cal["available"].map({"t": True, "f": False})




In [ ]:
cutoff = cal["date"].max() - pd.Timedelta(days=90)
recent = cal[cal["date"] >= cutoff]
recent["available"].isna().mean(), recent["date"].min(), recent["date"].max()


 




In [ ]:
# Hinweis: Calendar hat die Spalte 'listing_id'
unavail = (recent.groupby("listing_id")["available"]
                 .apply(lambda s: 1 - s.mean())
                 .rename("unavailable_rate_90d"))
unavail.head()


In [ ]:
# Listings-Tabelle hat 'id' als Schlüssel
merged = listings.merge(unavail, left_on="id", right_index=True, how="left")
merged["unavailable_rate_90d"] = merged["unavailable_rate_90d"].fillna(0.0)  # falls kein Calendar-Verlauf
merged.shape, merged[["price_eur","unavailable_rate_90d"]].head(3)


In [ ]:
merged["unavailable_rate_90d"].describe()
merged[["price_eur","unavailable_rate_90d"]].corr()


In [ ]:
OUT = "../data/processed/merged.csv"
merged.to_csv(OUT, index=False)
OUT


- Demand-Proxy = Anteil der Tage in den letzten 90 Tagen, an denen das Listing NICHT verfügbar war.
- Caveat: "unavailable" enthält sowohl gebuchte als auch host-blockierte Tage.
